In [1]:
# !pip install geocoder tqdm
# tqdm tells us how long it will take for our .apply to run
from tqdm.auto import tqdm
tqdm.pandas()

     |████████████████████████████████| 98 kB 6.4 MB/s eta 0:00:011
     |████████████████████████████████| 76 kB 5.2 MB/s eta 0:00:011
     |████████████████████████████████| 829 kB 13.1 MB/s eta 0:00:01
Using legacy 'setup.py install' for future, since package 'wheel' is not installed.
    Running setup.py install for future ... done
You should consider upgrading via the '/Users/ilenapeng/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import requests
import pandas as pd
pd.set_option("display.max_columns", None)

df = pd.read_csv("high_schools.csv")
df.head()

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,dbn,school_name,boro,phone_number,school_email,grade_span_min,grade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,school_type,overview_paragraph,program_highlights,language_classes,extracurricular_activities,addtl_info1,addtl_info2,start_time,end_time,ell_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10
0,12X550,High School of World Cultures,Bronx,7188608120,NaN,9,12,"Bx11, Bx27, Bx36, Bx4, Bx4A, Q44",6 to Elder Ave,1300 BOYNTON AVENUE,BRONX,NY,10472,www.highschoolofworldcultures.org,410,School for New Arrivals,Our motto is _Children First _ No Excuses _ Hi...,"AfterSchool programs, College Now","French, Spanish",Latin and African Dance,iLearnNYC Program for expanded online coursewo...,"Uniform Required HSWC uniform shirt, black sla...",80000.0,40000.0,"ESL, Dual Language Spanish",Eligibility Open only to New York City residen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04M372,ESPERANZA PREPARATORY ACADEMY,Manhattan,2127226507,wmerced2schools.nyc.gov,6,12,"M1, M101, M102, M106, M116, M15, M15SBS, M2, M...",6 to 110th St,240 EAST 109 STREET,MANHATTAN,NY,10029,httpschools.nyc.govSchoolPortals04M372,507,NaN,Esperanza Preparatory Academy EPA is a duallan...,Dual Language program Career and college couns...,Spanish,"Art, Community Service, Dance, Music, Student ...",iLearnNYC Program for expanded online coursewo...,Uniform Required Summer School Orientation Req...,80000.0,30000.0,"ESL, Dual Language Spanish",Eligibility Open only to heritage Spanish spea...,Priority to continuing 8th graders,Then to Manhattan students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN
2,28Q284,York Early College Academy,Queens,7182628547,28Q284schools.nyc.gov,6,12,"Q111, Q112, Q114, Q42, Q5, Q60",NaN,10835 167 STREET,QUEENS,NY,11433,httpschools.nyc.govSchoolPortals28Q284,611,NaN,The mission of York Early College Academy is t...,Early College,Spanish,"Chess, Art, Culture, Dance, Debate Team, Drama...",NaN,"Dress Code Required white polo shirt, blue jeans",80000.0,22000.0,ESL,Priority to continuing 8th graders,Then to Queens students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27Q351,Rockaway Collegiate High School,Queens,7187343290,nfennelschools.nyc.gov,9,12,"Q22, Q52, Q53, QM17","A, S to Beach 98th StPlayland",10000 BEACH CHANNEL DRIVE,QUEENS,NY,11694,httpschools.nyc.govSchoolPortals27Q351,397,NaN,Rockaway Collegiate High School RCHS provides ...,"Academic Program College Preparatory Classes, ...","French, Spanish","Chess, STEM Club, Drama, Internships, Music, C...",Academic Portfolio required Community Service ...,Community School Offers personalized learning ...,80000.0,30000.0,ESL,Priority to Queens students or residents who a...,Then to New York City residents who attend an ...,Then to Queens students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN
4,27Q262,Channel View School for Research,Queens,7186341970,Dharper22schools.nyc.gov,6,12,"Q22, Q52, Q53, QM17","A, S to Beach 98th StPlayland",10000 BEACH CHANNEL DRIVE,QUEENS,NY,11694,www.cvsr.info,683,NaN,"As an Expeditionary Learning school, our appro...",College Preparation that includes Advanced Pla...,Spanish,"Art and Design, Chess, Chorus, Computer, Inter...",iLearnNYC Program for expanded online coursewo...,Dress Code Required light blue or white collar...,81500.0,30400.0,ESL,Priority to continuing 8th graders,Then to District 27 students or residents who ...,Then to Queens students or residents who atten...,Then to New York City residents who attend an ...,Then to District 27 students or residents,Then to Queens students or residents,Then to New York City residents,NaN,NaN,NaN


In [3]:
API_KEY = 'o9rSOkoCojSt9SDZ85f9qAJMtGepjZYnHBmKasTljk0'

In [9]:
# https://geocode.search.hereapi.com/v1/geocode?q=5+Rue+Daunou%2C+75000+Paris%2C+France&apikey=YOUR_API_KEY

params = {
    'q': '1300 BOYNTON AVENUE BRONX NY 10472',
    'apikey': API_KEY
}

response = requests.get('https://geocode.search.hereapi.com/v1/geocode', params=params)
data = response.json()

if len(data['items']) > 0:
    print("Got a match")
    match = data['items'][0]    
    print(match['position'])
else:
    print("No matches")

# data

Got a match
{'lat': 40.83263, 'lng': -73.87835}


In [7]:
data['items'][0]['position']

{'lat': 40.83263, 'lng': -73.87835}

In [14]:
df['address'] = df.primary_address_line_1 + ' ' + df.city + ' ' + df.state_code + ' ' + df.zip.astype(str)
df.address.head()

0           1300 BOYNTON AVENUE  BRONX NY 10472
1       240 EAST 109 STREET  MANHATTAN NY 10029
2             10835 167 STREET  QUEENS NY 11433
3    10000 BEACH CHANNEL DRIVE  QUEENS NY 11694
4    10000 BEACH CHANNEL DRIVE  QUEENS NY 11694
Name: address, dtype: object

In [27]:
def geocode(address):
    print("I am going to geocode", address)

    params = {
        'q': address,
        'apikey': API_KEY
    }

    response = requests.get('https://geocode.search.hereapi.com/v1/geocode', params=params)
    data = response.json()
    if len(data['items']) > 0:
        match = data['items'][0]
        # Create a dictionary of everything important
        result = {
            'lat': match['position']['lat'],
            'lng': match['position']['lng']
        }
        
        # And then send it back!
        return pd.Series(result)
    else:
        print("No matches")
    
# if you use head, put it in front so that it runs faster (it'll pull the head and THEN geocode it)
# geocoded = df.head().address.apply(geocode)

# use tqdm to get the time
geocoded = df.address.progress_apply(geocode)

geocoded

  0%|          | 0/437 [00:00<?, ?it/s]

I am going to geocode 1300 BOYNTON AVENUE  BRONX NY 10472
I am going to geocode 240 EAST 109 STREET  MANHATTAN NY 10029
I am going to geocode 10835 167 STREET  QUEENS NY 11433
I am going to geocode 10000 BEACH CHANNEL DRIVE  QUEENS NY 11694
I am going to geocode 10000 BEACH CHANNEL DRIVE  QUEENS NY 11694
I am going to geocode 10000 BEACH CHANNEL DRIVE  QUEENS NY 11694
I am going to geocode 100 AMSTERDAM AVENUE  MANHATTAN NY 10023
I am going to geocode 100 ESSEX DRIVE  STATEN ISLAND NY 10314
I am going to geocode 100 ESSEX DRIVE  STATEN ISLAND NY 10314
I am going to geocode 100 HESTER STREET  MANHATTAN NY 10002
I am going to geocode 100 LUTEN AVENUE  STATEN ISLAND NY 10312
I am going to geocode 100 TRINITY PLACE  MANHATTAN NY 10006
I am going to geocode 100 WEST MOSHOLU PARKWAY SOUTH  BRONX NY 10468
I am going to geocode 100 WEST MOSHOLU PARKWAY SOUTH  BRONX NY 10468
I am going to geocode 100 WEST MOSHOLU PARKWAY SOUTH  BRONX NY 10468
I am going to geocode 10101 ROCKAWAY BOULEVARD  QUEE

I am going to geocode 207 01 116TH AVENUE  QUEENS NY 11411
I am going to geocode 2116 44TH ROAD  QUEENS NY 11101
I am going to geocode 2127 HIMROD STREET  QUEENS NY 11385
I am going to geocode 215 HEYWARD STREET  BROOKLYN NY 11206
I am going to geocode 215 WEST 114 STREET  MANHATTAN NY 10026
I am going to geocode 215 WEST 114 STREET  MANHATTAN NY 10026
I am going to geocode 220 HENRY STREET  MANHATTAN NY 10002
I am going to geocode 2225 WEBSTER AVENUE  BRONX NY 10457
I am going to geocode 223 EAST 23 STREET  MANHATTAN NY 10010
I am going to geocode 223 GRAHAM AVENUE  BROOKLYN NY 11206
I am going to geocode 223 GRAHAM AVENUE  BROOKLYN NY 11206
I am going to geocode 223 GRAHAM AVENUE  BROOKLYN NY 11206
I am going to geocode 225 WEST 24 STREET  MANHATTAN NY 10011
I am going to geocode 226 BRISTOL STREET  BROOKLYN NY 11212
I am going to geocode 226 BRISTOL STREET  BROOKLYN NY 11212
I am going to geocode 227243 WEST 61ST STREET  MANHATTAN NY 10023
I am going to geocode 23017 HILLSIDE AVENUE

I am going to geocode 400 PENNSYLVANIA AVENUE  BROOKLYN NY 11207
I am going to geocode 40 IRVING PLACE  MANHATTAN NY 10003
I am going to geocode 40 IRVING PLACE  MANHATTAN NY 10003
I am going to geocode 40 IRVING PLACE  MANHATTAN NY 10003
I am going to geocode 40 IRVING PLACE  MANHATTAN NY 10003
I am going to geocode 40 IRVING PLACE  MANHATTAN NY 10003
I am going to geocode 411 EAST 76 STREET  MANHATTAN NY 10021
I am going to geocode 411 PEARL STREET  MANHATTAN NY 10038
I am going to geocode 411 PEARL STREET  MANHATTAN NY 10038
I am going to geocode 411 PEARL STREET  MANHATTAN NY 10038
I am going to geocode 411 PEARL STREET  MANHATTAN NY 10038
I am going to geocode 4143 THIRD AVENUE  BRONX NY 10457
I am going to geocode 4200 16 AVENUE  BROOKLYN NY 11204
I am going to geocode 420 EAST 12 STREET  MANHATTAN NY 10009
I am going to geocode 425 WEST 123 STREET  MANHATTAN NY 10027
I am going to geocode 425 WEST 33RD STREET  MANHATTAN NY 10001
I am going to geocode 439 WEST 49TH STREET  MANHAT

I am going to geocode 911 FLATBUSH AVENUE  BROOKLYN NY 11226
I am going to geocode 9130 METROPOLITAN AVENUE  QUEENS NY 11375
I am going to geocode 9130 METROPOLITAN AVENUE  QUEENS NY 11375
I am going to geocode 921 EAST 228TH STREET  BRONX NY 10466
I am going to geocode 921 EAST 228TH STREET  BRONX NY 10466
I am going to geocode 921 EAST 228TH STREET  BRONX NY 10466
I am going to geocode 925 ASTOR AVENUE  BRONX NY 10469
I am going to geocode 925 ASTOR AVENUE  BRONX NY 10469
I am going to geocode 925 ASTOR AVENUE  BRONX NY 10469
I am going to geocode 925 ASTOR AVENUE  BRONX NY 10469
I am going to geocode 925 ASTOR AVENUE  BRONX NY 10469
I am going to geocode 928 SIMPSON STREET  BRONX NY 10459
I am going to geocode 9406 104TH STREET  QUEENS NY 11416
I am going to geocode 9425 117TH STREET  QUEENS NY 11419
I am going to geocode 9450 159 STREET  QUEENS NY 11433
I am going to geocode 965 LONGWOOD AVENUE  BRONX NY 10459
I am going to geocode 965 LONGWOOD AVENUE  BRONX NY 10459
I am going to 

,lat,lng
0,40.83263,-73.87835
1,40.79260,-73.94154
2,40.69692,-73.78641
3,40.58615,-73.82502
4,40.58615,-73.82502
...,...,...
432,40.69264,-73.86891
433,40.87726,-73.91258
434,40.87726,-73.91258
435,40.87726,-73.91258


In [23]:
geocoded = df.head().address.apply(geocode)
geocoded

I am going to geocode 1300 BOYNTON AVENUE  BRONX NY 10472
I am going to geocode 240 EAST 109 STREET  MANHATTAN NY 10029
I am going to geocode 10835 167 STREET  QUEENS NY 11433
I am going to geocode 10000 BEACH CHANNEL DRIVE  QUEENS NY 11694
I am going to geocode 10000 BEACH CHANNEL DRIVE  QUEENS NY 11694


,lat,lng
0,40.83263,-73.87835
1,40.79260,-73.94154
2,40.69692,-73.78641
3,40.58615,-73.82502
4,40.58615,-73.82502


In [24]:
merged = df.join(geocoded)
merged.head()

,dbn,school_name,boro,phone_number,school_email,grade_span_min,grade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,school_type,overview_paragraph,program_highlights,language_classes,extracurricular_activities,addtl_info1,addtl_info2,start_time,end_time,ell_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,address,lat,lng
0,12X550,High School of World Cultures,Bronx,7188608120,NaN,9,12,"Bx11, Bx27, Bx36, Bx4, Bx4A, Q44",6 to Elder Ave,1300 BOYNTON AVENUE,BRONX,NY,10472,www.highschoolofworldcultures.org,410,School for New Arrivals,Our motto is _Children First _ No Excuses _ Hi...,"AfterSchool programs, College Now","French, Spanish",Latin and African Dance,iLearnNYC Program for expanded online coursewo...,"Uniform Required HSWC uniform shirt, black sla...",80000.0,40000.0,"ESL, Dual Language Spanish",Eligibility Open only to New York City residen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1300 BOYNTON AVENUE BRONX NY 10472,40.83263,-73.87835
1,04M372,ESPERANZA PREPARATORY ACADEMY,Manhattan,2127226507,wmerced2schools.nyc.gov,6,12,"M1, M101, M102, M106, M116, M15, M15SBS, M2, M...",6 to 110th St,240 EAST 109 STREET,MANHATTAN,NY,10029,httpschools.nyc.govSchoolPortals04M372,507,NaN,Esperanza Preparatory Academy EPA is a duallan...,Dual Language program Career and college couns...,Spanish,"Art, Community Service, Dance, Music, Student ...",iLearnNYC Program for expanded online coursewo...,Uniform Required Summer School Orientation Req...,80000.0,30000.0,"ESL, Dual Language Spanish",Eligibility Open only to heritage Spanish spea...,Priority to continuing 8th graders,Then to Manhattan students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,240 EAST 109 STREET MANHATTAN NY 10029,40.79260,-73.94154
2,28Q284,York Early College Academy,Queens,7182628547,28Q284schools.nyc.gov,6,12,"Q111, Q112, Q114, Q42, Q5, Q60",NaN,10835 167 STREET,QUEENS,NY,11433,httpschools.nyc.govSchoolPortals28Q284,611,NaN,The mission of York Early College Academy is t...,Early College,Spanish,"Chess, Art, Culture, Dance, Debate Team, Drama...",NaN,"Dress Code Required white polo shirt, blue jeans",80000.0,22000.0,ESL,Priority to continuing 8th graders,Then to Queens students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10835 167 STREET QUEENS NY 11433,40.69692,-73.78641
3,27Q351,Rockaway Collegiate High School,Queens,7187343290,nfennelschools.nyc.gov,9,12,"Q22, Q52, Q53, QM17","A, S to Beach 98th StPlayland",10000 BEACH CHANNEL DRIVE,QUEENS,NY,11694,httpschools.nyc.govSchoolPortals27Q351,397,NaN,Rockaway Collegiate High School RCHS provides ...,"Academic Program College Preparatory Classes, ...","French, Spanish","Chess, STEM Club, Drama, Internships, Music, C...",Academic Portfolio required Community Service ...,Community School Offers personalized learning ...,80000.0,30000.0,ESL,Priority to Queens students or residents who a...,Then to New York City residents who attend an ...,Then to Queens students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,10000 BEACH CHANNEL DRIVE QUEENS NY 11694,40.58615,-73.82502
4,27Q262,Channel View School for Research,Queens,7186341970,Dharper22schools.nyc.gov,6,12,"Q22, Q52, Q53, QM17","A, S to Beach 98th StPlayland",10000 BEACH CHANNEL DRIVE,QUEENS,NY,11694,www.cvsr.info,683,NaN,"As an Expeditionary Learning school, our appro...",College Preparation that includes Advanced Pla...,Spanish,"Art and Design, Chess, Chorus, Computer, Inter...",iLearnNYC Program for expanded online coursewo...,Dress Code Required light blue or white collar...,81500.0,30400.0,ESL,Priority to continuing 8th graders,Then to District 27 students or residents who ...,Then to Queens students or residents who atten...,Then to New York City residents who attend an ...,Then to District 27 students or residents,Then to Queens students or residents,Then to New York City residents,NaN,NaN,NaN,1